## 1.创建docker，这个镜像中自带tensorrt环境
### -  sudo docker run --name=liu --gpus all -t -p 8888:8888 -v /home/yuzhenchen:/workspace nvcr.io/nvidia/pytorch:23.03-py3

- https://catalog.ngc.nvidia.com/orgs/nvidia/containers/pytorch

## `nvcr.io/nvidia/pytorch`

- `docker run -it --gpus=all --rm nvcr.io/nvidia/pytorch nvidia-smi`
    - 默认会去找 latest 的 tag，
    - 如果想指定版本 `docker run -it --gpus=all --rm nvcr.io/nvidia/pytorch:23.03-py3 nvidia-smi`
    - 指定 gpu id
        - `docker run -it --gpus="1" --rm nvcr.io/nvidia/pytorch:23.03-py3 nvidia-smi`
    - `-p 8888:8888`
        - 端口号映射，主机(宿主)端口:容器端口
    - `-v $PWD:/workspace`
        - 磁盘映射，volume
        - 这个 docker 镜像进去之后默认的路径就是 `~/workspace`
    - `--rm`:
        - 这个命令启动之后，会给这个 image（镜像）创建一个容器实例（container instance）
        - exit 后删除该启动的 container；

## 容器的 stop/start

### run vs. start

- run: create a new container of an image, and execute the container. 
    - docker run 的对象是 image id，而不是 container id
- start: Launch a **container** previously stopped. 

### stop/start/exec

- `docker stop $CONTAINER_ID`
- `docker start $CONTAINER_ID`
- `docker exec -it $CONTAINER_ID bash`
- `docker rm $CONTAINER_ID`

## `-p` 与 `ssh`（终端机与server）

- `-p` 主机(宿主)端口:容器端口
    - `-p 8887:8888`
- `ssh -L`
    - `ssl -L 8886:localhost:8887 server`

## 2.TensorRT(yolov5的特detect为例)
- tensorrt的GitHub：https://github.com/wang-xinyu/tensorrtx/tree/master/yolov5

### `需要在yolov5文件下编译，会用到该文件下的c++文件，如果在别处运行tensorrt，需要复制yolov5的文件夹，从头生成文件再编译，重新生成libmyplugins.so文件`

### 生成模型之前要修改yolov5/src/config.h中的18行：类别数=自己的类别数

- 使用gen_wts.py文件生成wts格式的模型
    - python gen_wts.py -w yolov5s.pt -o yolov5s.wts  # -w:输入pt模型路径， -o：输出wts模型路径
- 新建build文件夹，把上一步生成的wts模型复制到build文件夹中,然后编译
    - mkdir build && cd build
    - cp path/yolov5s.wts . 
    - cmake ..
    - make
- 使用生成的yolov5_det来生成engine模型
    - ./yolov5_det -s yolov5s.wts yolov5s.engine s  # 最后的s表示使用的s级v5模型，如果是v5m则把s要换成m
- 使用生成的engine模型测试
    - ./yolov5_det -d yolov5s.engine ../images
- 在python中使用engine模型检测
    - python yolov5_det_trt.py  # yolov5_det_trt_cuda_python.py文件会报错：超出索引
    